In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [3]:
#Load dataset

titles = pd.read_csv("Listing_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
dev_df = pd.read_csv("Train_Tagged_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)


In [4]:
#fill the missing values in the Tag column with ffill
dev_df['Tag'].fillna(method='ffill', inplace=True)

/var/folders/7w/8xc_pk4n6zg1zdkphtnj8r5c0000gn/T/ipykernel_45246/2685090846.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dev_df['Tag'].fillna(method='ffill', inplace=True)


In [5]:
dev_df.head()

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,Produktlinie
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ


In [6]:
#check for missing values
dev_df.isnull().sum()

Record Number    0
Title            0
Token            0
Tag              0
dtype: int64

In [7]:
#Concatenate the Title and Token columns in dev_df
dev_df["Token_Title"] = dev_df["Title"] + " " + dev_df["Token"]
dev_df.head()


,Record Number,Title,Token,Tag,Token_Title
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell,Supreme Nike SB Dunk High By any Means Red US1...
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke,Supreme Nike SB Dunk High By any Means Red US1...
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie,Supreme Nike SB Dunk High By any Means Red US1...
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,Produktlinie,Supreme Nike SB Dunk High By any Means Red US1...
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ,Supreme Nike SB Dunk High By any Means Red US1...


In [8]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split

X = dev_df["Token_Title"]
y = dev_df["Tag"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)


In [9]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
#tokenize the training and testing sets
train_tokens = X_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
test_tokens = X_test.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

#pad the tokens to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#pad the tokens to the maximum length of all the tokens

train_tokens = pad_sequences(train_tokens, maxlen=50, dtype="long", value=0, truncating="post", padding="post")
test_tokens = pad_sequences(test_tokens, maxlen=50, dtype="long", value=0, truncating="post", padding="post")

print(train_tokens.shape)


(41387, 64)


In [29]:
#convert the tokens to tensors and obtain embeddings <CRASHES KERNEL>

train_tokens = torch.tensor(train_tokens)
test_tokens = torch.tensor(test_tokens)

with torch.no_grad():
    train_embeddings = model(train_tokens)
    test_embeddings = model(test_tokens)
    
train_embeddings = train_embeddings[0][:,0,:].numpy()
test_embeddings = test_embeddings[0][:,0,:].numpy()